In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/DATASET.zip /content/DATASET.zip

In [ ]:
!unzip /content/DATASET.zip

In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial import distance as dist
import os
import cv2
import imutils
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.models import load_model
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [ ]:
images = []
targets = []

image_size = (64, 64)

# Folder paths for categories
training_paths = [
    '/content/DATASET/TRAIN/IMAGES/EOSINOPHIL/',
    '/content/DATASET/TRAIN/IMAGES/LYMPHOCYTE/',
    '/content/DATASET/TRAIN/IMAGES/MONOCYTE/',
    '/content/DATASET/TRAIN/IMAGES/NEUTROPHIL/'
]

testing_paths = [
    '/content/DATASET/TEST/IMAGES/EOSINOPHIL/',
    '/content/DATASET/TEST/IMAGES/LYMPHOCYTE/',
    '/content/DATASET/TEST/IMAGES/MONOCYTE/',
    '/content/DATASET/TEST/IMAGES/NEUTROPHIL/'
]

In [ ]:
def findedges(image):
    gray = cv2.GaussianBlur(image, (1, 1), 0)
    edged = cv2.Canny(gray, 100, 400)
    edged = cv2.dilate(edged, None, iterations=1)
    edged = cv2.erode(edged, None, iterations=1)
    return edged

def getimageconturs(edged):
    contours = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)
    contours = sorted(contours, key=lambda x: cv2.contourArea(x))
    return contours

def getboxes(contours,orig):
    boxes = []
    centers = []
    for contour in contours:
        box = cv2.minAreaRect(contour)
        box = cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box)
        box = np.array(box, dtype="int")
        (tl, tr, br, bl) = box
        if (np.linalg.norm(tl - bl)) > 0 and (np.linalg.norm(tl - tr)) > 0:
            boxes.append(box)
    return boxes

In [ ]:
code={"EOSINOPHIL":0,"LYMPHOCYTE":1,"MONOCYTE":2,"NEUTROPHIL":3}
def getcode(n):
    if type(n)==str:
        for x,y in code.items():
            if n==x:
                return y
    else:
        for x,y in code.items():
            if n==y:
                return x

In [ ]:
def preprocess_load_data(folder_paths):
    # Iterate through the folders
    for label, folder_path in enumerate(folder_paths):
        image_files = os.listdir(folder_path)
        for image_file in image_files:
            # Load and preprocess the image
            image_path = os.path.join(folder_path, image_file)
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # add padding to the image to better detect cell at the edge
            image = cv2.copyMakeBorder(image,10,10,10,10,cv2.BORDER_CONSTANT,value=[198, 203, 208])

            #thresholding the image to get the target cell
            image1 = cv2.inRange(image,(80, 80, 180),(180, 170, 245))

            # openning errosion then dilation
            kernel = np.ones((3, 3), np.uint8)
            kernel1 = np.ones((5, 5), np.uint8)
            img_erosion = cv2.erode(image1, kernel, iterations=2)
            image1 = cv2.dilate(img_erosion, kernel1, iterations=5)

            #detecting the blood cell
            edgedImage = findedges(image1)
            edgedContours = getimageconturs(edgedImage)
            edgedBoxes =  getboxes(edgedContours, image.copy())

            if len(edgedBoxes)==0:
                continue
            # get the large box and get its cordinate
            last = edgedBoxes[-1]
            max_x = int(max(last[:,0]))
            min_x = int( min(last[:,0]))
            max_y = int(max(last[:,1]))
            min_y = int(min(last[:,1]))

            # draw the contour and fill it
            mask = np.zeros_like(image)
            cv2.drawContours(mask, edgedContours, len(edgedContours)-1, (255,255,255), -1)

            # any pixel but the pixels inside the contour is zero
            image[mask==0] = 0

            # extract th blood cell
            image = image[min_y:max_y, min_x:max_x]

            if (np.size(image)==0):
                continue
            # resize th image
            image = cv2.resize(image, image_size)
            image = image / 255.0  # Normalize pixel values to [0, 1]

            # Create a custom target vector based on requirements
            # Example: [1, 0, 0, 1] for the first and fourth categories
            if label == 0:
                target = [1,1,0,0,0,0,1,0,0,0]
            elif label == 1:
                target = [0,0,0,1,0,0,0,0,0,1]
            elif label == 2:
                target = [0,0,0,0,1,0,0,1,1,0]
            elif label == 3:
                target = [1,0,0,0,0,1,0,0,0,0]

            images.append(image)
            targets.append(target)

    return images, targets

In [ ]:
images, targets = preprocess_load_data(folder_paths=training_paths)
print(len(images), len(targets))

9720 9720


In [ ]:
test_images, test_targets = preprocess_load_data(folder_paths=testing_paths)
print(len(images), len(targets))

12146 12146


In [ ]:
images = np.array(images)
targets = np.array(targets)

In [ ]:
images,targets=shuffle(images,targets,random_state=10)

In [ ]:
train_image,test_image,train_label,test_label=train_test_split(images,targets,test_size=.2)

In [ ]:
print(images.shape)
print(targets.shape)
print(train_image.shape)
print(test_image.shape)

(12146, 64, 64, 3)
(12146, 10)
(9716, 64, 64, 3)
(2430, 64, 64, 3)


In [ ]:
# Function to create the custom deep learning model
def create_custom_model(input_shape=(64,64,3), num_output_units=10):
    model = keras.models.Sequential()

    # Convolutional layers
    model.add(keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape))
    model.add(keras.layers.MaxPooling2D((2, 2)))
    model.add(keras.layers.Dropout(.25))

    model.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 2)))
    model.add(keras.layers.Dropout(.25))

    model.add(keras.layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 2)))
    model.add(keras.layers.Dropout(.25))

    # Flatten the output
    model.add(keras.layers.Flatten())

    # Dense layers
    model.add(keras.layers.Dense(512, activation='relu'))
    model.add(keras.layers.Dropout(.25))

    model.add(keras.layers.Dense(1024, activation='relu'))
    model.add(keras.layers.Dropout(.25))

    model.add(keras.layers.Dense(num_output_units, activation='sigmoid'))

    return model

In [ ]:
from sklearn.model_selection import KFold

k_folds = 10
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

accuracy_scores = []

for train_index, val_index in kf.split(images):
    # Split the data into training and validation sets
    X_train, X_val = images[train_index], images[val_index]
    y_train, y_val = targets[train_index], targets[val_index]

    # Create and compile your Keras model
    model = create_custom_model()
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train, epochs=30, batch_size=32, verbose=1)

    # Evaluate the model on the validation set
    _, accuracy = model.evaluate(X_val, y_val)
    accuracy_scores.append(accuracy)

print("Average Accuracy: {:.2f}%".format(np.mean(accuracy_scores) * 100))

Epoch 1/30
342/342 [==============================] - 13s 14ms/step - loss: 0.3435 - accuracy: 0.5731
Epoch 2/30
342/342 [==============================] - 5s 13ms/step - loss: 0.1354 - accuracy: 0.6819
Epoch 3/30
342/342 [==============================] - 5s 15ms/step - loss: 0.0920 - accuracy: 0.6830
Epoch 4/30
342/342 [==============================] - 4s 13ms/step - loss: 0.0799 - accuracy: 0.6996
Epoch 5/30
342/342 [==============================] - 4s 13ms/step - loss: 0.0641 - accuracy: 0.6800
Epoch 6/30
342/342 [==============================] - 5s 15ms/step - loss: 0.0544 - accuracy: 0.7058
Epoch 7/30
342/342 [==============================] - 5s 14ms/step - loss: 0.0480 - accuracy: 0.7072
Epoch 8/30
342/342 [==============================] - 5s 13ms/step - loss: 0.0447 - accuracy: 0.7009
Epoch 9/30
342/342 [==============================] - 5s 15ms/step - loss: 0.0501 - accuracy: 0.7102
Epoch 10/30
342/342 [==============================] - 5s 13ms/step - loss: 0.0363 - accur

KeyboardInterrupt: 

In [ ]:
# Define the number of output units (elements in the output vector)
num_output_units = len(targets[0])  # Should match the length of your custom target vectors

# Create the custom model
input_shape = images[0].shape
model = create_custom_model(input_shape, num_output_units)
print(input_shape)

# Compile the model with mean squared error loss
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

(64, 64, 3)


In [ ]:
# Train the model
history = model.fit(
    train_image,          # Input images
    train_label,         # Custom target vectors
    epochs=30,       # Number of training epochs
    batch_size=32,   # Batch size
    validation_split=0.1
)

In [ ]:
loss,accuracy=model.evaluate(test_image, test_label)
print("the accuracy of test image is : ",accuracy)

In [ ]:
model_file = 'vector_predictor_v64.h5'
model.save(model_file)

In [ ]:
loaded_model = load_model('/content/drive/MyDrive/BCCD_Models/vector_predictor_v64.h5')
loss,accuracy=loaded_model.evaluate(test_image, test_label)
print("the accuracy of test image is : ",accuracy)

76/76 [==============================] - 5s 8ms/step - loss: 0.0026 - accuracy: 0.9444
the accuracy of test image is :  0.9444444179534912


In [ ]:
predicted_vectors = loaded_model.predict(test_image)
print(predicted_vectors[0])

76/76 [==============================] - 0s 4ms/step
[9.9998999e-01 1.0227482e-03 4.1250522e-10 5.5750843e-06 2.4503991e-06
 9.9858701e-01 8.9392281e-04 1.0677670e-06 3.2355701e-06 1.5351559e-05]


In [ ]:
predicted_vectors = predicted_vectors.tolist()
predicted_vectors = [element for sublist in predicted_vectors for element in sublist]
actual_vectors = [element for sublist in test_label for element in sublist]
actual_vectors = np.array(actual_vectors)
predicted_vectors = np.array(predicted_vectors)
print(actual_vectors.shape, predicted_vectors.shape)
predicted_vectors = np.round(predicted_vectors).astype(int)

(24300,) (24300,)


In [ ]:
report = classification_report(actual_vectors, predicted_vectors)
accuracy = accuracy_score(actual_vectors, predicted_vectors)
conf_matrix = confusion_matrix(actual_vectors, predicted_vectors)

print(report)
print(f"Accuracy: {accuracy}")
print("\nConfusion Matrix:")
print(conf_matrix)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18297
           1       0.99      0.99      0.99      6003

    accuracy                           1.00     24300
   macro avg       1.00      1.00      1.00     24300
weighted avg       1.00      1.00      1.00     24300

Accuracy: 0.9969135802469136

Confusion Matrix:
[[18259    38]
 [   37  5966]]
